# Transform Demand data, so it can be used by a Neural Network

In [25]:
import datetime
import pandas as pd

from typing import Tuple

## Generate Demand Data and Passenger objects

In [21]:
from src.robin.demand.entities import Demand, Passenger

path_config_demand = '../configs/test_case/demand_data.yml'

demand = Demand.from_yaml(path_config_demand)

passengers = demand.generate_passengers()

## Get passenger relevant information and save it in a dataframe

In [30]:
def get_passenger_info(passenger: Passenger) -> Tuple[str, str, str, datetime.datetime]:
    """
    Get the information of a passenger and return it as a tuple

    Args:
        passenger: Passenger object

    Returns:
        Tuple[str, str, str, datetime.date, float]: Tuple with the information of the passenger
    """
    user_pattern = passenger.user_pattern.name
    origin, destination = passenger.market.departure_station, passenger.market.arrival_station
    arrival_day = datetime.datetime.combine(passenger.arrival_day.date, datetime.datetime.min.time())
    arrival_float = passenger.arrival_time
    arrival_hour = int(arrival_float)
    arrival_minute = int((arrival_float - arrival_hour) * 60)
    arrival_time = arrival_day + datetime.timedelta(hours=arrival_hour, minutes=arrival_minute)
    return user_pattern, origin, destination, arrival_time

# Map a list of Passenger objects to a list of tuples with the passenger information
passengers_info = list(map(get_passenger_info, passengers))

# Dataframe with the passenger information
df = pd.DataFrame(passengers_info, columns=['user_pattern', 'origin', 'destination', 'arrival_time'])

print(df.head())

  user_pattern origin destination        arrival_time
0     Business  60000       04040 2023-06-01 08:48:00
1     Business  60000       04040 2023-06-01 09:24:00
2     Business  60000       04040 2023-06-01 06:25:00
3     Business  60000       04040 2023-06-01 08:38:00
4     Business  60000       04040 2023-06-01 07:57:00


In [31]:
print(passengers_info[0])

('Business', '60000', '04040', datetime.datetime(2023, 6, 1, 8, 48))


# Embedding of the user patterns, origins and destinations

Test Keras word2vec implementation from: https://sites.nd.edu/munira-syed/2019/10/25/word2vec-implementation-with-keras-2-0/

In [33]:
from keras.layers import dot
from keras.layers.core import Dense, Reshape
from keras.layers import Embedding
from keras.models import Sequential
from keras.models import Model

vocab_size = 3
embed_size = 5

word_model = Sequential()
word_model.add(Embedding(vocab_size, embed_size,
               embeddings_initializer="glorot_uniform",
               input_length=1))
word_model.add(Reshape((embed_size, )))

context_model = Sequential()
context_model.add(Embedding(vocab_size, embed_size,
                  embeddings_initializer="glorot_uniform",
                  input_length=1))
context_model.add(Reshape((embed_size,)))

dot_product = dot([word_model.output, context_model.output], axes=1,
                  normalize=False)
dot_product = Dense(1, kernel_initializer="glorot_uniform",
              activation="sigmoid")(dot_product)
model = Model(inputs=[word_model.input, context_model.input],
              outputs=dot_product)

model.compile(loss="mean_squared_error", optimizer="rmsprop")

2023-04-24 14:10:46.328363: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
